In [1]:
# vLLM itself
!pip install vllm

# Hugging Face transformers for model handling
!pip install transformers

# BitsAndBytes (for quantization support like 4-bit/8-bit)
!pip install bitsandbytes

# Accelerate (required for model device mapping)
!pip install accelerate
# vLLM itself
!pip install vllm

# Hugging Face transformers for model handling
!pip install transformers

# BitsAndBytes (for quantization support like 4-bit/8-bit)
!pip install bitsandbytes

# Accelerate (required for model device mapping)
!pip install accelerate

# For caching models and logging
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 MB 3.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 74.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 47.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 1.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 2.0 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 6.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 6.7 MB/s eta 0:00:00:00:01

In [2]:
import torch

# Check GPU status
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Total Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"Available Memory: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated(0)) / 1e9:.2f} GB")

# Clear cache
torch.cuda.empty_cache()

GPU: Tesla T4
Total Memory: 15.83 GB
Available Memory: 15.83 GB


In [ ]:
import os
from huggingface_hub import login

# Load token from environment variable
hf_token = os.getenv("HF_TOKEN")
login(token=hf_token)

In [4]:
import os
os.environ['VLLM_USE_V1'] = '0'

import torch
import gc

torch.cuda.empty_cache()
gc.collect()

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        with torch.cuda.device(i):
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()

import pandas as pd
from vllm import LLM, SamplingParams
from tqdm import tqdm

df = pd.read_csv('/kaggle/input/banglachq/train.csv')
df = df[['id', 'question']].copy()
print(f"Loaded {len(df)} rows")

model_name = "google/gemma-3-1b-it"
print(f"\nLoading {model_name}...")

llm = LLM(
    model=model_name,
    trust_remote_code=True,
    dtype="float32",  # 1B should work with float16 on T4
    gpu_memory_utilization=0.7,
    max_model_len=2048,
    max_num_seqs=24,
)

tokenizer = llm.get_tokenizer()
print("Model loaded!\n")

def translate_to_chittagonian(bangla_text):
    prompt = f"""You are a precise translation tool. Your only task is to translate the given Bangla text to Chittagonian (Chatgaiyan) dialect using Bengali script.

    INSTRUCTIONS:
    - Translate the Bangla text below to Chittagonian (Chatgaiyan) dialect
    - Use only Bengali script (not Latin script or IPA)
    - Return ONLY the translated text with no additional commentary, explanations, or notes
    - Do not include phrases like "Here is the translation:" or "The Chittagonian translation is:"
    - Do not add any metadata, formatting, or extra information
    - If you cannot translate a specific word, keep it as is in the original form

    Here are some examples of Bangla to Chittagonian translations:
    Bangla: খালু, গ্রামের বাড়ি যাবেন কবে
    Chittagonian: অহালু, গ্রামর বাড়িত হত্তে যাইবেন?

    Bangla: আপনি কি নিয়মিত জল খান?
    Chittagonian: অনে কি পইত্তেন পানি হন না?

    Bangla: জুম্মার পর যাব।
    Chittagonian: জুমর পর যাইয়ুম।

    Bangla: পালং শাক খুব পছন্দ
    Chittagonian: পালংশাক বেশি গম লাগে দে

    Bangla: বাঙালি নানা-নানীরা কি করেন?
    Chittagonian: বাআলর নানানানী অলে কি গরে দে?

    Bangla text to translate:
    {bangla_text}

    Chittagonian translation:"""  
    
    messages = [{"role": "user", "content": prompt}]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

sampling_params = SamplingParams(
    temperature=0.3,
    top_p=0.9,
    max_tokens=512,
    stop=["Bangla:"]
)

BATCH_SIZE = 24
all_translations = []

for batch_start in tqdm(range(0, len(df), BATCH_SIZE), desc="Translating"):
    batch_end = min(batch_start + BATCH_SIZE, len(df))
    batch_prompts = [translate_to_chittagonian(row['question']) for _, row in df.iloc[batch_start:batch_end].iterrows()]
    
    outputs = llm.generate(batch_prompts, sampling_params)
    all_translations.extend([o.outputs[0].text.strip() for o in outputs])
    
    del outputs, batch_prompts
    gc.collect()
    torch.cuda.empty_cache()

df['chittagonian_translation'] = all_translations
df.to_csv('gemma3_1b_translations.csv', index=False)
print("Done!")

2025-09-27 19:43:51.672551: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759002232.042957      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759002232.154005      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


INFO 09-27 19:44:10 [__init__.py:216] Automatically detected platform cuda.
Loaded 1880 rows

Loading google/gemma-3-1b-it...
INFO 09-27 19:44:12 [utils.py:328] non-default args: {'trust_remote_code': True, 'dtype': 'float32', 'max_model_len': 2048, 'gpu_memory_utilization': 0.7, 'max_num_seqs': 24, 'disable_log_stats': True, 'model': 'google/gemma-3-1b-it'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

INFO 09-27 19:44:28 [__init__.py:742] Resolved architecture: Gemma3ForCausalLM


...

Translating:  99%|█████████▊| 78/79 [15:28<00:10, 10.79s/it]

Adding requests:   0%|          | 0/8 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/8 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Translating: 100%|██████████| 79/79 [15:33<00:00, 11.82s/it]

Done!
